In [1]:
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau, TensorBoard
from keras.models import Model
import keras

Using TensorFlow backend.


In [2]:
import os

path = '/home/hamza/Documents/Hamza-git/data17'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.cpp' in file:
            files.append(os.path.join(r, file))

# for f in files:
#     print(f)

In [3]:
len(files)

56837

In [4]:
outputvar = {}
counter = 0
for x in files:
    user = x.split('/')[-2]
    if user not in outputvar.keys():
        outputvar[user] = counter
        counter += 1

In [5]:
lengths = []

In [6]:
def padding(fname):
    f = open(fname, "r")
    y = f.read()
    f.close()
    
    origy = y
    while (len(y) < 500):
        y += origy
        
    f = open(fname, "w")
    f.write(y)
    f.close()
    return y

In [8]:
for i in range(0,3):
    lengths = []
    count = 0
    for fil in files:
        f = open(fil, "r")
        x = f.read()
        f.close()
        if(len(x) <= 100):
            files.remove(fil)
        elif len(x) == 0:
            print(fil)
            files.remove(fil)
        elif(len(x) < 500):
            x = padding(fil)
            lengths.append(len(x))
        else:
            lengths.append(len(x))

In [9]:
print(max(lengths))
print(min(lengths))
print(len(lengths))
print(len(files))

95319
500
56808
56808


In [10]:
import numpy as np

In [12]:
inp = np.array(files)
names = []
for fil in files:
    names.append(outputvar[fil.split('/')[-2]])
out = np.array(names)
print(inp[:10])
print(out[:10])

['/home/hamza/Documents/Hamza-git/data17/farhit/A1.cpp'
 '/home/hamza/Documents/Hamza-git/data17/farhit/B1.cpp'
 '/home/hamza/Documents/Hamza-git/data17/farhit/D0.cpp'
 '/home/hamza/Documents/Hamza-git/data17/farhit/C0.cpp'
 '/home/hamza/Documents/Hamza-git/data17/farhit/B2.cpp'
 '/home/hamza/Documents/Hamza-git/data17/farhit/C1.cpp'
 '/home/hamza/Documents/Hamza-git/data17/farhit/B0.cpp'
 '/home/hamza/Documents/Hamza-git/data17/farhit/A0.cpp'
 '/home/hamza/Documents/Hamza-git/data17/chemthan/practice2.cpp'
 '/home/hamza/Documents/Hamza-git/data17/chemthan/practice0.cpp']
[0 0 0 0 0 0 0 0 1 1]


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(
inp, out, test_size=0.8, random_state = 42)

In [15]:
def walk(path, vocab, count):
    for r, d, f in os.walk(path):
        for file in f:
            f = open(os.path.join(r, file))
            x = f.read()
            for line in x:
                for letter in line:
                    if letter not in vocab.keys():
                        vocab[letter] = count[0]
                        count[0] += 1


In [16]:
vocab = {}
count = [0]
path = '/home/hamza/Documents/Hamza-git/data17/'
for r, d, f, in (os.walk(path)):
    for i, dir in enumerate(d):
        walk(os.path.join(r, dir), vocab, count)
print(count)

[1468]


In [17]:
def getFiles(path):
    matrix = []
    for p in path:
        tmp = np.zeros((50, 10), dtype = int)
        f = open(p)
        for i in range(0,50):
            for j in range(0, 10):
                tmp[i][j] = vocab[f.read(1)]
        matrix.append(tmp)
    return np.array(matrix)

In [18]:
getFiles(['/home/hamza/Documents/Hamza-git/data17/0bstacle/main0.cpp']).shape


(1, 50, 10)

In [19]:
num_classes = len(outputvar)
input_shape = (50, 10)
batch_size = 1000
def data_generator(file_paths, author_numbers):
    batch_start = 0
    batch_end = batch_start + batch_size
    n = file_paths.shape[0]
    indexes = np.arange(0, n, batch_size)
    if n % batch_size != 0:
        indexes = indexes[:-1] 
    while True:
        np.random.shuffle(indexes)
        for ind in indexes:
            batch_start = ind
            batch_end = batch_start + batch_size
            myFiles = file_paths[batch_start:batch_end]
            batch_files = getFiles(myFiles)
            batch_authors = author_numbers[batch_start:batch_end]
            batch_x = np.zeros((batch_size, 50, 10))
            tmp = np.zeros((batch_size, num_classes))
            batch_x = np.array( batch_files )
            for i, j in enumerate(batch_authors):
                tmp[i][j] = 1
            batch_y = np.array( tmp )
            yield( batch_x, batch_y )

In [20]:
input_fil = Input(shape=(input_shape), name='input_fil')
conv = Conv1D(8, kernel_size=3, strides = 2,  activation='relu')(input_fil)
maxP = MaxPooling1D()(conv)
flat = Flatten()(maxP)
output_class = Dense(num_classes, activation='softmax')(flat)

model = Model(inputs=input_fil, outputs=output_class)

In [21]:
model_name = 'Code-Attribution'

if not os.path.exists('./'+model_name):
    os.mkdir(model_name)
    
adam = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', 
              optimizer=adam, 
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_fil (InputLayer)       (None, 50, 10)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 24, 8)             248       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 12, 8)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 96)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 11609)             1126073   
Total params: 1,126,321
Trainable params: 1,126,321
Non-trainable params: 0
_________________________________________________________________


In [22]:
class LRTensorBoard(TensorBoard):
    def __init__(self, *args, **kwargs):
        super(LRTensorBoard, self).__init__(*args, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        logs.update({'lr': K.eval(self.model.optimizer.lr)})
        super(LRTensorBoard, self).on_epoch_end(epoch, logs)

In [23]:
epochs = 25
    
checkpoint = ModelCheckpoint(model_name+'/'+model_name+'-{epoch:02d}-{val_loss:.2f}.h5', 
                             monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
cvslogger = CSVLogger(model_name+'/logs.csv', separator=',', append=True)
#reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.000001)
tensorboard = LRTensorBoard(log_dir='./'+model_name, histogram_freq=0, write_graph=True, write_grads=1, 
                            batch_size=batch_size, write_images=True)

callbacks = [checkpoint, tensorboard, cvslogger]

In [24]:
train_gen = data_generator(X_train, Y_train)
test_gen = data_generator(X_test, Y_test)


In [ ]:
hist1 = model.fit_generator(train_gen, epochs=epochs, steps_per_epoch=len(Y_train)//batch_size, 
                           validation_data=test_gen, validation_steps=len(Y_test)//batch_size, 
                           callbacks=callbacks, verbose=1)

Epoch 1/25
 3/11 [=======>......................] - ETA: 23s - loss: 0.0011 - acc: 0.9999